<i><b>Public-AI</b></i>
#### week 1. 고객 데이터와 개인화 추천시스템

# Section 1. 비개인화 추천 시스템

개인화 추천 시스템이 부상하기 전에도 추천시스템은 존재했습니다. 그 중 대표적인 방법이 '순위'를 이용한 비개인화 추천입니다. 누가 어디서 무엇을 구매했느냐 등의 정보를 복합적으로 고려하는 개인화 추천시스템과 다르게 비개인화 추천은 주어진 데이터 안에서 인기도나 최신성 등의 통계치를 내어 정렬을 하고 상위의 상품을 노출합니다. 이번에는 본격적으로 개인화 추천시스템을 배우기에 앞서 비개인화된 방식으로 추천할 아이템을 찾아봅시다. <br>

### _Objective_ 
* [데이터베이스의 데이터 확인하기] CSV 파일을 통해 데이터를 확인하는 방법을 배웁니다.
* [데이터 탐색하기] Pandas를 이용해 데이터를 탐색하고, 원하는 형태에 따라 데이터를 변형하는 방법을 배웁니다. 
* [비개인화 추천: 아이템의 순위 매기기] 비개인화 추천시스템 중 대표적인 방법인 순위를 활용한 추천 방식을 배웁니다.

In [ ]:
# 필요한 라이브러리 가져오기
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.utils import get_file

# 영화 포스터를 가져와 주피터에서 볼 수 있도록 만들어주는 메소드
def display_posters(movieId_list):
    import requests
    from io import BytesIO
    from PIL import Image
    
    def get_poster(movieId):
        url = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens/img/POSTER_20M_FULL/{}.jpg".format(movieId)
        try:
            response = requests.get(url)
            b = BytesIO(response.content)
            img = np.asarray(Image.open(b))
        except:
            img = np.zeros((200,100,3))
        return img
    
    images = []
    for i in movieId_list:
        img = get_poster(i)
        images.append(img)

    columns = 5
    rows = (len(movieId_list) - 1) // columns + 1
    fig, axes = plt.subplots(rows, columns)
    axes = axes.flatten()
    fig.set_size_inches((20,rows * 5))
        
    for ind, img in enumerate(images):
        c_id = ind % columns
        r_id = ind // columns
        ax = axes[r_id*columns + c_id]
        ax.set_title(movieId_list[ind])
        ax.imshow(img)

    plt.tight_layout()

# \[ 데이터 확인하기 \]
---

## 1. CSV를 통해 데이터를 가져오기

지금 데이터에는 총 3가지 Dataframe으로 구성되어 있습니다. 영화에 대한 정보가 담겨진 `movie_df`, 영화 별 장르에 대한 정보가 담겨진 `genre_df`, 영화에 대한 별점 정보가 담겨진 `rating_df`으로 구성되어 있습니다.

In [ ]:
ROOT_URL = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens/datasets"

movie_path = get_file("movies.csv", os.path.join(ROOT_URL, "movies.csv"))
movie_df = pd.read_csv(movie_path)

genre_path = get_file("genres.csv", os.path.join(ROOT_URL, "genres.csv"))
genre_df = pd.read_csv(genre_path)

rating_path = get_file("ratings.csv", os.path.join(ROOT_URL, "ratings.csv"))
rating_df = pd.read_csv(rating_path)

## 2. Movie-Lens 데이터베이스의 테이블 확인하기


### (1) movie dataframe 확인하기


#### 데이터 일부 가져오기

#### 전체 행 갯수 확인하기

#### 해당 데이터의 컬럼 정보 가져오기

### (2) rating dataframe 확인하기

#### 데이터 일부 가져오기

#### 전체 행 갯수 확인하기

#### 해당 데이터의 컬럼 정보 가져오기

####  주요 통계 정보 가져오기

### (3) genre dataframe 확인하기

#### 일부 가져오기 

####  전체 행 갯수 확인하기

####  해당 데이터의 컬럼 정보 가져오기

# \[ 데이터 탐색하기 \]
---

Pandas는 파이썬의 대표적인 데이터 분석 패키지입니다. Pandas는 '코드로 치는 엑셀'과 비슷하다고 생각하면 됩니다. 코드로서 테이블에서 필요한 정보들을 탐색하고, 계산하고, 시각화하는 기능을 제공해줍니다.<br>

## 1. 각 테이블에 대한 주요 통계정보 확인하기

### (1) `movies` 테이블  탐색하기


#### 데이터 내 영화의 연도 범위 살펴보기 

#### 출시 연도 별 영화 갯수 가져오기

출시 연도별로 얼마나 많은 영화의 데이터가 있는지 살펴보기 위해 `groupby` 구문을 배워보도록 하겠습니다. `groupby` 는 어떤 기준을 중심으로 데이터를 합쳐 집계 함수를 적용하고자 할 때 사용됩니다.<br>

#### Bar 그래프를 통해  출시 연도 별 영화 갯수 시각화하기

그래프를 보면, 1990년대 이후 영화의 평점이 많고, 최근인 2014년도와 2015년도 영화의 평점이 아직 많이 쌓이지 않았음을 알 수 있습니다. 

### (2) `genres` 테이블 탐색하기

`genres` 테이블을 간단히 탐색해 보겠습니다. 

#### 장르 별 영화의 갯수를 가져오기

아까와 같이 가져오되, 내림차순으로 정렬하여 가져오도록 해보겠습니다.

#### PIE 그래프를 통해  출시 연도 별 장르 별 영화의 갯수 시각화하기

#### 영화 별 장르의 갯수를 가져오기

아까와 같이 가져오되, 내림차순으로 정렬하여 가져오도록 해보겠습니다.

#### 히스토그램 그래프를 통해  영화 별 장르의 수를 시각화하기

영화 별 장르의 수를 히스토그램으로 나타내면 아래와 같습니다. 대부분 하나 혹은 두 개의 장르 정보를 가지는 것을 알 수 있습니다. 

### (3) `ratings` 테이블 탐색하기

이번에는 `ratings` 데이터를 확인해봅시다.

#### 시간 정보를 `datetime` 포맷으로 변환하기

여기에서 `rate_at`은 시간을 나타내는 방법 중 하나로, 1970년 1월 1일부터 몇 초가 지났는지를 계산한 결과입니다.`rated_at`를 시간값으로 변환하기 위해서는`pd.to_datetime()`을 이용하면 됩니다.

#### 시간 정보를 연도로 변환하기

타임스탬프 값을 datetime으로 변경해주면 아래와 같이 시간 계산을 매우 편리하게 수행할 수 있습니다. `.dt.year`로 지칭하여 연도만 가져올 수 있습니다.

#### 연도 별 별점 수 구하기

#### 연도 별점 수를 시각화하기 

#### 유저 별로 평균 선호도 계산하기

#### 유저 별 평균 선호도를 Histogram으로 시각화하기

# \[ 비개인화 추천 : 아이템의 순위 매기기 \]
---

가장 간단한 형태의 추천은 **잘 나가는 제품 혹은 서비스**을 제시하는 것입니다. 가장 평이 좋은 제품 혹은 가장 구매가 빈번한 제품, 최근에 나온 제품 등을 추천하는 것은 고객에 대한 정보가 전혀 없을 때 매우 유용합니다. 그렇기 때문에 대부분의 커머스 사이트에서 랜딩 페이지에 인기 순위 혹은 신상을 소개하는 경우가 많습니다. Pandas의 정렬함수와 집계함수를 활용해 (1) 최신 아이템, (2) 최고 평점 아이템, (3) 최근 인기 아이템 리스트를 만들어보겠습니다.<br>

## 1. 최신(신규) 아이템 추천하기

> 최근에 나온 제품 있나요?

아이템의 출시일(발매일)을 기준으로 정렬을 하여, 가장 최근에 등장한 것들 위주로 가져올 수 있습니다. 영화의 경우 출시시점에 따라 최신 순으로 나열할 수 있을 것입니다. `movies`데이터의 `release_year` 칼럼이 있으므로, 이를 기준으로 상위 10개 아이템을 뽑아보겠습니다. 

## 2. 최고 평점 아이템 추천하기
> 평이 제일 좋은 물건은 무엇인가요?

평균 평점을 이용해 평이 제일 좋은 물건을 뽑아서 보여줄 수도 있을 것입니다. 다만, 평점이 높은 아이템을 뽑을 때는 한개, 두개의 평만 있는 것들은 제외시킬 필요가 있습니다. 충분히 많은 사람들로부터 검증받지 못한 것들은 그 평을 신뢰할 수 없기 때문입니다.  보통 응답조사를 할 때 300명 이상을 유의미하다고 판단하기 때문에, 300명 이상 평가를 받은 영화들만 추려내도록 하겠습니다. <br>

#### 300개 이상 평점을 받은 영화 중 Top 10을 선정하기

#### 300개 이상 평점을 받은 **Comedy** 영화 중 Top 10 선정하기

장르 별로 최고 평점 영화들을 뽑아서 보여준다면, 특정 장르에 관심이 많은 유저에게 유용한 정보를 제공할 수 있을 것입니다. 이번에는 코미디 영화만 추려서 동일한 방법으로 최고 평점 영화를 뽑아봅시다. 

## 3. 최근 인기 아이템(HOT) 추천하기

> 요즘 사람들이 제일 많이 찾는 것이 무엇인가요?

영화의 평점이 높은 순으로 영화를 추천하게 되면 어떤 약점이 있을까요? 오래된 영화일수록 평가가 많이 쌓이게 되기 때문에, 과거 평가 기준으로 영화를 추천하게 됩니다. 최근에 개봉한 영화는 아직 평점이 많지 않아 추천되지 않을 수 있습니다. 최신 영화를 위주로 추천하고 싶다면, 아래와 같은 방식으로 최근 개봉영화만을 추려내어 추천을 해야 합니다. <br>

#### 최근 4년 간 출시되었던 영화 정보 가져오기 

#### 최근 4년간 영화 중 Top 10 선정하기

`hot10_movie_ids` 영화를 기준으로 앞서 인기 영화를 뽑은 기준을 그대로 적용하여 최근 인기 아이템을 뽑아보겠습니다. 결과를 확인해보면, 앞서 전체 데이터에서 인기 영화를 뽑았을 때에는 보이지 않았던 최신 영화가 리스트됨을 알 수 있습니다. 

#  

---

    Copyright(c) 2020 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2020/03/30


---